### Парсинг данных и создание датафрейма

In [ ]:
import os
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

# %matplotlib notebook

In [ ]:
runN = "*" # If take all runs set runN = "*"
paths = glob.glob(f"build/Run{runN}_nt_ParticleData_t*.csv")

In [ ]:
with open(paths[0], 'r') as f:
    print(f"Headers from file: {paths[0]}")
    hlines = []
    while True:
        line = f.readline()
        if len(line) == 0 or line[0] != '#':
            break
        hlines.append(line)
    title = hlines[1].replace('#title ', '')[:-1]
    sep = chr(int(hlines[2].replace('#separator ', '')[:-1]))
    vsc_sep = chr(int(hlines[3].replace('#vector_separator ', '')[:-1]))
    columns = [' '.join(l.split(' ')[2:])[:-1] for l in hlines[4:]]
    meta_len = len(hlines)
    print(f'Metarows: {meta_len}\nColumns: {columns}')

In [ ]:
data = pd.DataFrame()
for path in paths:
    item = pd.read_csv(path, skiprows=meta_len, names=columns)
    data = data.append(item, ignore_index=True)

In [ ]:
data

In [ ]:
set(data["particle_name"])

In [ ]:
len(data[data["particle_name"]=="e-"])

In [ ]:
len(data[data["particle_name"]=="gamma"])

### Отобразим собранные данные

In [ ]:
fig = plt.figure(figsize = (6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data[data["particle_name"]=="gamma"]["X"][:1500],
           data[data["particle_name"]=="gamma"]["Y"][:1500],
           data[data["particle_name"]=="gamma"]["Z"][:1500], marker=".")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()

### Поворот распределения для более удобной работы

In [ ]:
xyz_e = data[data["particle_name"]=="gamma"]
xdata = xyz_e["X"].to_numpy()
ydata = xyz_e["Y"].to_numpy()
zdata = xyz_e["Z"].to_numpy()
xyz = np.array([xdata, ydata, zdata]).T

In [ ]:
# Матрица поворота вокруг оси х
ang = -np.pi/4 # 45 градусов
Mx = [[1, 0, 0], [0, np.cos(ang), -np.sin(ang)], [0, np.sin(ang), np.cos(ang)]]

In [ ]:
xyz = np.array([np.matmul(point, Mx) for point in xyz])

### Преобразование к полярным координатам

In [ ]:
# Я это писал 6 часов, поскольку не думал что формулы из вики или матанализа не работают
def cart2sph(n_xyz: np.array, trh=10**-9):
    ra, θa, φa = [], [], []
    for x, y, z in n_xyz:
        r = np.sqrt(x**2 + y**2 + z**2)
        θ = 0. if np.abs(r) < trh else np.arccos(z/r)
        if x>0 and y>0:
            φ = np.arctan(y/x)
        elif np.abs(x) < trh and y>0:
            φ = np.pi/2
        elif x<0 and y>0:
            φ = np.pi + np.arctan(y/x)
        elif x<0 and y<0:
            φ = np.pi + np.arctan(y/x)
        elif np.abs(x) < trh and y<0:
            φ = np.pi*3/4
        elif x>0 and y<0:
            φ = 2*np.pi + np.arctan(y/x)
        elif np.abs(x) < trh and np.abs(y) < trh:
            φ = 0.
        ra.append(r)
        θa.append(θ)
        φa.append(φ)
    return np.array([ra, θa, φa]).T

def sph2cart(n_rθφ: np.array):
    n_rθφ[:, 2] = n_rθφ[:, 2] % (2*np.pi)
    n_rθφ[:, 1] = n_rθφ[:, 1] % (np.pi)
    x = n_rθφ[:, 0] * np.sin(n_rθφ[:, 1]) * np.cos(n_rθφ[:, 2])
    y = n_rθφ[:, 0] * np.sin(n_rθφ[:, 1]) * np.sin(n_rθφ[:, 2])
    z = n_rθφ[:, 0] * np.cos(n_rθφ[:, 1])
    return np.array([x, y, z]).T

In [ ]:
nrθφ = cart2sph(xyz)

In [ ]:
print(f"θ range: {min(nrθφ[:,1])} to {max(nrθφ[:,1])}")
print(f"φ range: {min(nrθφ[:,2])} to {max(nrθφ[:,2])}")

### Выбор интересующей области

In [ ]:
θ_from = np.pi/2-np.pi/10
θ_to = np.pi/2+np.pi/10.

In [ ]:
segment, out_segment = [], []
i = 0
j = 0
for r, theta, phi in nrθφ:
    if θ_from<theta and theta<θ_to:
        segment.append((1, theta, phi))
        i += 1
    else:
        out_segment.append((1, theta, phi))
        j += 1
    
xyz_segment = sph2cart(np.array(segment))
xyz_out_segment = sph2cart(np.array(out_segment))

In [ ]:
fig = plt.figure(figsize = (6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*xyz_segment[:200].T, c="green", marker=".")
ax.scatter(*xyz_out_segment[:800].T, marker=".")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_zlim(-1, 1)
plt.show()

In [ ]:
segment_theta = np.array(segment).T[2] # Select TPhi
hist = np.histogram(segment_theta, bins=100)
segment_theta.shape

In [ ]:
theta = hist[1][:100]
radii = hist[0]

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(theta, radii)
ax.set_title("Диаграмма направленности рентгеновского излучения.", va='bottom')
plt.show()

### Диаграмма по шаровому поясу

In [ ]:
xyz_e = data[data["particle_name"]=="gamma"]
xdata = xyz_e["X"].to_numpy()
ydata = xyz_e["Y"].to_numpy()
zdata = xyz_e["Z"].to_numpy()
endata = xyz_e["energy"].to_numpy()
xyz = np.array([xdata, ydata, zdata]).T

In [ ]:
ang = -np.pi/2 # 45 градусов
def rotate(nxyz: np.array, axis: str, ang):
    assert nxyz.shape[1] == 3 # shape == (N, 3)
    Mx = [[1, 0, 0], [0, np.cos(ang), -np.sin(ang)], [0, np.sin(ang), np.cos(ang)]]
    My = [[np.cos(ang), 0, np.sin(ang)], [0, 1, 0], [-np.sin(ang), 0, np.cos(ang)]]
    Mz = [[np.cos(ang), -np.sin(ang), 0], [np.sin(ang), np.cos(ang), 0], [0, 0, 1]]
    M = {"X":Mx, "Y":My, "Z":Mz}
    return np.array([np.matmul(point, M[axis]) for point in nxyz])

In [ ]:
xyz = rotate(xyz, "X", np.pi)

In [ ]:
xyz.shape

In [ ]:
fig = plt.figure(figsize = (6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xyz[:1500,0],
           xyz[:1500,1],
           xyz[:1500,2], marker=".")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()

Подсчёт по кольцам

In [ ]:
nrθφ = cart2sph(xyz)

In [ ]:
segments = [], []
dθ = np.pi/12
for r, theta, phi in nrθφ:
    if θ_from<theta and theta<θ_to:
        segment.append((1, theta, phi))
    else:
        out_segment.append((1, theta, phi))

xyz_segment = sph2cart(np.array(segment))
xyz_out_segment = sph2cart(np.array(out_segment))